In [1]:
import findspark # You will need to pip install findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "MyShell") # Spark UI at http://localhost:4040/jobs/


In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [50]:
sources = "file:///Users/jim/DEV/pyspark_notebooks/resources/test_cases/100/Test10*_*.*"
df = sqlContext.read.json(sources)

In [51]:
df.printSchema()

root
 |-- events: struct (nullable = true)
 |    |-- DeliveryLocationArrival: struct (nullable = true)
 |    |    |-- reasonCode: string (nullable = true)
 |    |    |-- sourceType: string (nullable = true)
 |    |-- GeolocationUpdate: struct (nullable = true)
 |    |    |-- fromAddress: struct (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- countryCode: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- postalCode: string (nullable = true)
 |    |    |    |-- stateCode: string (nullable = true)
 |    |    |    |-- street: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- reasonCode: string (nullable = true)
 |    |    |-- sourceType: string (nullable = true)
 |    |    |-- toAddress: struct (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- countryCode: string (nullable = t

In [75]:
shipments = df['paired']['shipmentId']

In [76]:
for s in shipments:
    print s

TypeError: Column is not iterable

In [58]:
s = df.filter(df['paired']['shipmentId'] == 'Test101|%RUN_TS%').sort(df['timestamp']['asOf'])

In [63]:
s.count()

16

In [68]:
s.first().asDict(True)

{'events': {'DeliveryLocationArrival': None,
  'GeolocationUpdate': None,
  'PickUpLocationDeparture': None,
  'ShipmentDeliveryEstimate': None,
  'ShipmentEnd': None,
  'ShipmentStart': {'legCount': 1,
   'legs': [{'carrierAlphaCode': u'LAIR',
     'etaAlertThreshold': 60,
     'etaNoticeThreshold': 30,
     'etaSubscriber': u'true',
     'etaWarningThreshold': 15,
     'fromPoiId': 1,
     'fromPoiName': u'SAVI TECHNOLOGY HQ [SAVI]',
     'geoDriftThreshold': 50,
     'geoWarpThreshold': 200,
     'legNumber': 1,
     'mode': u'truck',
     'toPoiId': 2,
     'toPoiName': u'REAGAN NATIONAL AIRPORT [DCA]',
     'toStopNum': 1,
     'vesselId': u'JH4TB2H26CC000000'}],
   'parties': {'carriers': [u'LAIR'],
    'customers': [u'SAVI', u'VADOT'],
    'divisions': [u'US', u'VA', u'DC'],
    'suppliers': [u'TESTCO']},
   'reasonCode': u'goodsIssued',
   'referenceIds': [u'Test101|%RUN_TS%', u'Package01|%RUN_TS%'],
   'shipmentId': u'Test101|%RUN_TS%',
   'shippingPointFences': u'eyJjcnMiOiB7

In [69]:
s.last().asDict(True)

AttributeError: 'DataFrame' object has no attribute 'last'

In [23]:
x = shipments.join(carriers, shipments.shipment == carriers.shipment)

In [31]:
s = shipments
s.crosstab("origin", "destination").show()

+------------------+---+----+---+---+
|origin_destination|BWI|SAVI|IAD|DCA|
+------------------+---+----+---+---+
|               BWI|  0|   1|  0|  0|
|               IAD|  0|   1|  0|  0|
|               DCA|  1|   2|  1|  0|
|              SAVI|  1|   0|  1|  2|
+------------------+---+----+---+---+



In [33]:
s.describe().show()

+-------+-----------------+-----------------+------------------+
|summary|          pallets|         shipment|             value|
+-------+-----------------+-----------------+------------------+
|  count|               10|               10|                10|
|   mean|             44.2|            146.2|           72221.2|
| stddev|32.67278854201323|15.58346702259945|21604.252121592486|
|    min|                1|              123|             28564|
|    max|              100|              165|             99620|
+-------+-----------------+-----------------+------------------+



In [39]:
s.groupBy('origin').avg('pallets', 'value').show()

+------+------------+----------+
|origin|avg(pallets)|avg(value)|
+------+------------+----------+
|   DCA|        25.5|  76985.75|
|  SAVI|        43.0|   75412.0|
|   BWI|       100.0|   84057.0|
|   IAD|        68.0|   28564.0|
+------+------------+----------+



In [60]:
x.groupBy('carrier').avg('value').sort('avg(value)').show()

+-------+-----------------+
|carrier|       avg(value)|
+-------+-----------------+
|   DART|          46498.0|
|   USXI|          73002.0|
|   YODA|          75418.0|
|   SCNN|85652.66666666667|
+-------+-----------------+

